In [367]:
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport
from pkg_resources import ContextualVersionConflict

# from rich.console import Console

import requests
import json

import time
from sqlalchemy import delete
from tqdm import tqdm
import re

# console = Console()

sample_transport=RequestsHTTPTransport(
    url='http://localhost:4001/graphql',
    use_json=True,
    headers={
        "Content-type": "application/json",
    },
    verify=False
)

client = Client(
    #execute_timeout=20,
    transport=sample_transport,
    fetch_schema_from_transport=True,
)

session = requests.Session()
session.auth = ('admin', "secret")

#hostname = 'http://ec2-3-64-237-95.eu-central-1.compute.amazonaws.com:8080/tilt/tilt'
#auth = session.post(hostname)
#response = json.loads(session.get(hostname).content)

#tilt = json.loads(requests.get('http://ec2-3-64-237-95.eu-central-1.compute.amazonaws.com:8080/tilt/tilt').content)
#print("Fetch Successful!", response)


json_file = open('/Users/johannes/Desktop/TU/playground/tilt')
tilt = json.load(json_file)

In [398]:

def create_node(parent):

    """
    func:   create_node
            Recursively goes through JSON and builds a mutation to generate the neo4j graph
            Base case is when an entry is a string
    """

    def rem_dq(uncleaned):

        """
        func:   rem_dq
                removes double quotes. 
                Needed to avoid errors in upload
        """
        if isinstance(uncleaned,str):
            print("trying something")
            return re.sub('"', '', uncleaned)
        else:
            return uncleaned

    #mutation string
    label_string = ""

    #iterate through elements of dictionary or list
    for i, key in enumerate(parent):

        #if key == 'rightToRectificationOrDeletion':
            #label_string += " HERE "
            
        try:
            if key[0] == "$": # the dollar sign infront of the $oid is not recognised so we have to remove it
                label_string += f'{key[1:]}: "{rem_dq(parent[key])}"'


            # if key is a list
            elif isinstance(parent[key], list):

            
                #if list item is empty
                if len(parent[key]) == 0:
                    
                    #label_string += f'{key}: "{parent[key]}"'
                    label_string += "" # just add nothing. not ideal as we lose out on recording missing information but easier to fix than trying to fix missfilled fields in tilt

                #two types of entries: nodes or strings
                else:

                    # When we get lists with multiple same node entries, we make a list of nodes. 
                    # There we can only need one create statement and then close it afterwards
                    label_string += key+ ':{create:['
                    node_list = True

                    for list_item in parent[key]:
                        
                        # some entries are a string wrapped in a lists instead of a node.
                        if type(list_item) == type(""): 
                            #deleted = label_string[-len(f'{key}:{{create:['):]
                            label_string = label_string[:-len(f'{key}:{{create:[')]                             
                            label_string += f'{key}: "{rem_dq(list_item)}"'
                            node_list = False

                        # if the list items are dictionaries make a new node
                        else: 
                            label_string += f'{{{create_node(list_item)}}},'

                    #
                    if node_list:
                        label_string = label_string[:-1]+']}'
            
                
            # if the element is a dictionary
            elif isinstance(parent[key], dict): 
                
                #create the next node for this dict
                label_string += f'{key}:{{create:[{{{create_node(parent[key])}}}]}}'

            else: # base case
                
                if i < len(parent)-1:

                    label_string += f'{key}: "{rem_dq(parent[key])}", '
                    
                else:
                    label_string += f'{key}: "{rem_dq(parent[key])}"'
                
        except Exception as ex:
            label_string += "BIG ERROR HERE"  
            print(ex)  

    #certain characters (like tilda) are rejected by gql
    label_string = re.sub('~', "", label_string)
    label_string = label_string.strip().replace('\n', "")
    
    return label_string

    

In [399]:
print("mutation { createtilts( input: [ { " + create_node(tilt[2]) + "}] ) {tilts {meta { name } } }}")

isinstance() arg 2 must be a type or tuple of types
isinstance() arg 2 must be a type or tuple of types
isinstance() arg 2 must be a type or tuple of types
isinstance() arg 2 must be a type or tuple of types
isinstance() arg 2 must be a type or tuple of types
isinstance() arg 2 must be a type or tuple of types
isinstance() arg 2 must be a type or tuple of types
isinstance() arg 2 must be a type or tuple of types
isinstance() arg 2 must be a type or tuple of types
isinstance() arg 2 must be a type or tuple of types
isinstance() arg 2 must be a type or tuple of types
isinstance() arg 2 must be a type or tuple of types
isinstance() arg 2 must be a type or tuple of types
isinstance() arg 2 must be a type or tuple of types
isinstance() arg 2 must be a type or tuple of types
isinstance() arg 2 must be a type or tuple of types
isinstance() arg 2 must be a type or tuple of types
isinstance() arg 2 must be a type or tuple of types
isinstance() arg 2 must be a type or tuple of types
isinstance()

In [392]:
tilt_mutations = []
for tilt_entry in tqdm(tilt):
    tilt_entry  
    tilt_mutations.append("mutation { createtilts( input: [ { " + create_node(tilt_entry) + "}] ) {tilts {meta { name } } }}")

100%|██████████| 73/73 [00:00<00:00, 2177.74it/s]


In [393]:
successes = 0
for i, query_string in enumerate(tqdm(tilt_mutations)):
    #print("Query String: ", query_string)
    try:
        query = gql(query_string)
        client.execute(query)
        successes += 1
    except Exception as syn:
        print(syn)
        print("Error With Tilt No.", i)

print("Successfully Uploaded:", 100*(successes/(i+1)), "per. of tilts")

 52%|█████▏    | 38/73 [01:15<01:09,  1.97s/it]

Syntax Error: Invalid number, expected digit but got: 'A'.

GraphQL request:1:18073
1 | mutation { createtilts( input: [ { _id:{create:[{oid: "62559213e7dfaf542de42cdf"
  | }]}meta:{create:[{_id: "c55855c5-1972-401c-b9d9-b1e5c718c6f6", name: "paypal", c
  | reated: "2021-06-21T12:21:52.743500", modified: "2021-06-21T12:21:52.743520", ve
  | rsion: "1", language: "de", status: "active", url: "https://www.paypal.com/de/we
  | bapps/mpp/ua/privacy-full", _hash: "d187cde9ed0ee3e28044156101efc2468d081aa325be
  | ebed7685c38132414be6"}]}controller:{create:[{name: "PayPal", country: "Luxemburg
  | ", address: "None", division: " PayPal (Europe) S.a.r.l. et Cie", representative
  | :{create:[{name: "None", email: "None", phone: "None"}]}}]}dataProtectionOfficer
  | :{create:[{email: "None", phone: "None", country: "None", name: "None", address:
  |  "None"}]}dataDisclosed:{create:[{_id: "bac38eea-32d7-4d54-a2ca-baaf4570fbbb", c
  | ategory: "E-Mail-Adresse", purposes:{create:[{purpose: "Inform

 86%|████████▋ | 63/73 [01:57<00:17,  1.80s/it]

Syntax Error: Expected ':', found String ' ZahlungenVertragserfüllung30. Beratung per SMS Vertragserfüllung oder Einwilligung31. Generierung und Weiterleitung vonKündigungsschreiben an Vorversorger undVorversicherer im Rahmen desKündigungsservicesVertragserfüllung oder Einwilligung'.

GraphQL request:1:7051
1 | mutation { createtilts( input: [ { _id:{create:[{oid: "6255921be7dfaf542de42d59"
  | }]}meta:{create:[{_id: "e0374156-7bc1-4641-a4c2-f261edb1fb30", name: "verivox", 
  | created: "2021-06-21T12:21:52.431660", modified: "2021-06-21T12:21:52.431691", v
  | ersion: "1", language: "de", status: "active", url: "https://www.verivox.de/date
  | ien/datenschutzbestimmungen-verivox-gmbh-120378.pdf", _hash: "8481ee85fdf293168e
  | 0f88081d19dc794f1f6737b6808c18480c49d944ff7a4d"}]}controller:{create:[{name: "Ve
  | rivox GmbH", country: "69115", address: "Max-Jarecki-Straße 21", division: "None
  | ", representative:{create:[{name: "Maximilian Hartung", email: "service@verivox.
  | de", ph

 92%|█████████▏| 67/73 [02:13<00:21,  3.63s/it]

Syntax Error: Expected ':', found String ') '.

GraphQL request:1:18213
1 | mutation { createtilts( input: [ { _id:{create:[{oid: "6255921de7dfaf542de42d70"
  | }]}meta:{create:[{_id: "b2326270-15ad-4a3c-afbb-317b43c752c8", name: "webde", cr
  | eated: "2021-06-21T12:21:52.509774", modified: "2021-06-21T12:21:52.509794", ver
  | sion: "1", language: "de", status: "active", url: "https://agb-server.web.de/dat
  | enschutz", _hash: "db271845c6bdb0fe0d37195b1cafd2ec72566b87534e8f187099a2951a7fc
  | e27"}]}controller:{create:[{name: "1&amp;amp;amp;amp;amp;amp;1 Mail &amp;amp;amp
  | ;amp;amp;amp; Media GmbH", country: "None", address: "None", division: "None", r
  | epresentative:{create:[{name: "None", email: "None", phone: "None"}]}}]}dataProt
  | ectionOfficer:{create:[{email: "datenschutz@webde.de", phone: "None", country: "
  | None", name: "None", address: "None"}]}dataDisclosed:{create:[{_id: "a1e3bb55-e9
  | 49-4d98-8b3b-ddd84cb64b1c", category: "Namen und Anschrift", purposes:{cre

100%|██████████| 73/73 [02:54<00:00,  2.39s/it]

Successfully Uploaded: 95.8904109589041 per. of tilts


In [376]:
tilt[67]

{'_id': {'$oid': '6255921de7dfaf542de42d70'},
 'meta': {'_id': 'b2326270-15ad-4a3c-afbb-317b43c752c8',
  'name': 'webde',
  'created': '2021-06-21T12:21:52.509774',
  'modified': '2021-06-21T12:21:52.509794',
  'version': 1,
  'language': 'de',
  'status': 'active',
  'url': 'https://agb-server.web.de/datenschutz',
  '_hash': 'db271845c6bdb0fe0d37195b1cafd2ec72566b87534e8f187099a2951a7fce27'},
 'controller': {'name': '1&amp;amp;amp;amp;amp;amp;1 Mail &amp;amp;amp;amp;amp;amp; Media GmbH',
  'country': None,
  'address': None,
  'division': None,
  'representative': {'name': None, 'email': None, 'phone': None}},
 'dataProtectionOfficer': {'email': 'datenschutz@webde.de',
  'phone': None,
  'country': None,
  'name': None,
  'address': None},
 'dataDisclosed': [{'_id': 'a1e3bb55-e949-4d98-8b3b-ddd84cb64b1c',
   'category': 'Namen und Anschrift',
   'purposes': [{'purpose': [], 'description': []}],
   'legalBases': [{'reference': [], 'description': []}],
   'legitimateInterests': [{'reaso